![ ](https://scontent-arn2-2.xx.fbcdn.net/v/t1.0-9/57429716_2030860760555937_2750062083545497600_n.jpg?_nc_cat=100&ccb=2&_nc_sid=8bfeb9&_nc_ohc=y5d_WudgJy0AX9PNi5o&_nc_ht=scontent-arn2-2.xx&oh=a630ca82a715594c37e77c981b2f0b12&oe=6004F26F)
                                   


 1. EXPLORATORY DATA ANALYSIS
   
   - 1.1 Libraries
   
   - 1.2 Acquire the data
  
   - 1.3 Descriptive statistics
  
   - 1.4 Data visualisation
   

2. Feature Engineering

   - 2.1 Filling missing Values
  
   - 2.2 Binning the categorical features
  
   - 2.3 Creating New Features
   
   - 2.4 Removing irrelevant variables
   
   - 2.4 Creating dummy variables
   
  

3. Pre-Modeling Tasks


   -  3.1 Defining Features in Training/Test Set
   
   -  3.2 Splitting the dataset
   

4. Modeling
 
  
   - Random Forest Model
   
   
5. Evaluating the performance of the model
     
     - Confusion Matrix
     - Classificarion Report
     - Accuracy Score
     - Precision Score
     - ROC Curve
     
6. Submission


- Useful resources
  

# 1. EXPLORATORY DATA ANALYSIS


 ## Libraries 📚

In [ ]:

import numpy as np 
import pandas as pd 
import pandas_profiling
import matplotlib.pyplot as plt 
import plotly.express as px 
import seaborn as sns 
sns.set()

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder   
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

    
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report, balanced_accuracy_score



# Color Palette

custom_colors = ["#85CEDA","#D2A7D8", "#A67BC5", "#BB1C8B", "#05A4C0"]
customPalette = sns.set_palette(sns.color_palette(custom_colors))

# Set size

sns.palplot(sns.color_palette(custom_colors),size=1)
plt.tick_params(axis='both', labelsize=0, length = 0)

### 1.2 Acquire data

In [ ]:
train = pd.read_csv("../input/titanic/train.csv")
test  = pd.read_csv("../input/titanic/test.csv")

* **Variables** 

-`PassengerId` is the unique id of the row and it doesn't have any effect on target

- `Name` 

- `Sex` 

- `Age`

-`Survived` is the target variable we are trying to predict (0 or 1):
     
      1 = Survived
      0 = Not Survived
-`Pclass` (Passenger Class) is the socio-economic status of the passenger and it is a categorical ordinal feature which has 3 unique values (1, 2 or 3):
      
      1 = Upper Class
      2 = Middle Class
      3 = Lower Class

-`SibSp` is the total number of the passengers' siblings and spouse.

-`Parch` is the total number of the passengers' parents and children.

-`Ticket` is the ticket number of the passenger.

-`Fare` is the passenger fare.

-`Cabin` is the cabin number of the passenger.

-`Embarked` is port of embarkation and it is a categorical feature which has 3 unique values (C, Q or S):
      
     C = Cherbourg
     Q = Queenstown
     S = Southampton

### 1.3 Descriptive Statistics

* So first we need to know **which variables are available in the dataset** ?

In [ ]:
print (train.columns.values)

* **Categorical Features** : Survived,Embarked and Sex 
 Ordinal: Pclass.

* **Numerical Features** : Continous: Age, Fare. Discrete: SibSp, Parch.

* **What are the data types for various features?**

In [ ]:
train.info()
test.info()

* The next step we'll do some descriptive statistics, this one helps us to describe and understand the features of a specific data by giving short summaries about the sample and measures of the data.


* **Statistical info about the numerical variables** :

In [ ]:
train.describe()

In [ ]:
survived_summary = train.groupby("Sex")
survived_summary.mean().reset_index()

* Total samples are 891 or 40% of the actual number of passengers on board the Titanic (2,224).
* only 38% passenger survived 
* 74% female passenger survived, and only 19% male passenger survived.
* About 75% of passengers did not travel with their children or parents.
* Around 30% of the passengers had siblings aboard


* **Checking for the correlation**

In [ ]:
# Correlation Map

train.corr
f,ax = plt.subplots(figsize=(15,10))
sns.heatmap(train.corr(), annot =True, linewidth =".5", fmt =".2f", cmap=custom_colors)
plt.show()

* **pandas_profiling**

In [ ]:
profile = pandas_profiling.ProfileReport(train)

In [ ]:
profile

### 1.4 Data Visualization 📊📈

* **Survived feature**

Distribution of survivals : 1 is for survival and 0 is for not

In [ ]:
train["Survived"].value_counts()

In [ ]:
fig = px.bar(train.Survived.value_counts(), width=900, height=400)
fig.update_traces(marker_color='orchid')
fig.show()

* **Survived by Age**

In [ ]:
# BarPlot


plt.figure(figsize=(10,8))
plt.title("Survived people based on gender")
sns.barplot(x="Survived",y="Age", data =train,palette='mako_r')

* **Age**

In [ ]:
def hist(x,title):
    plt.figure(figsize = (10,8))
    ax = sns.distplot(x, 
                 kde=False);
    values = np.array([rec.get_height() for rec in ax.patches])
    norm = plt.Normalize(values.min(), values.max())
    colors = plt.cm.jet(norm(values))
    for rec, col in zip(ax.patches, colors):
        rec.set_color(col)
    plt.title(title)

In [ ]:
hist(train['Age'],'Distribution of Age')


* **Fare**

In [ ]:
hist(train['Fare'],'Distribution of Fare') 


* **Sex feature vs Survived feature**

In [ ]:
# BarPlot

# Set the width and the height of the figure
plt.figure(figsize=(12,8))

# Add the title
plt.title("Survived people based on gender")

# Draw a barplot of survival people by sex
sns.barplot(x="Sex",y="Survived", data =train,palette=(custom_colors))

# Print percentage of males vs females that are survived 
print("Percentage of females who survived :",  train["Survived"][train["Sex"] == 'female'].value_counts(normalize = True)[1]*100)
print ("Percentage of males who survived :",  train["Survived"][train["Sex"]==  'male'].value_counts(normalize= True)[1]*100)

In [ ]:
# swarmplot

plt.figure(figsize=(15,9))

sns.swarmplot(x=train['Age'], y=train['Sex'], hue='Survived', data =train,palette =custom_colors)

- According to this graph, we can notice that womens are more likely to survive.

* **PARCH feature vs Survived feature**

In [ ]:
# Barplot
plt.figure(figsize=(15,9))
sns.barplot(x="Parch", y="Survived", data = train, palette=custom_colors)
plt.show

- People with less than four parents or childrens aboard more likely to survive.

* **SibSp Feature** 

In [ ]:
# Barplot

plt.figure(figsize=(15,9))
sns.barplot(x="SibSp", y="Survived", data=train, palette= 'RdPu_r')
plt.show

- People with SibSp or spouses were less likely to survive, therefore people with no children were more less likely to survived than those with one children or two.

* **Embarked and fare features**

Let's now see how the embarkation site affects the survival.

In [ ]:
# Violinplot

fig = plt.figure(figsize=(25, 7))
sns.violinplot(x ="Embarked", y ="Fare", hue ="Survived", data=train, split =True , palette = {0: "#3498db", 1:"#2ecc71"});

* **Passenger Class**

In [ ]:
fig, ax = plt.subplots(figsize = (10,7))
ax = sns.countplot(x = 'Survived', hue = 'Pclass', data = train, palette = custom_colors)
ax.set_xlabel('Survived')
ax.set_title('Survival Rate for Passenger Classes', fontsize = 14, fontweight='bold');

## 2. Feature Engineering

* **Inspect the missing values** 

In [ ]:
# let's take more detaild look of what data is actually missing

total = train.isnull().sum().sort_values(ascending=False)
percent_1 = train.isnull().sum()/train.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(5)

* **Missing values in train data**

In [ ]:
train.isnull().any()

* **Missing values in test data**

In [ ]:
test.isnull().any()

### 2.1 Filling missing Values

In [ ]:
#Age


train["Age"] = train["Age"]. fillna(train["Age"].mean())

test["Age"]  = test["Age"] . fillna(test["Age"].mean())


#Fare

test["Fare"] = test ["Fare"]. fillna(test["Fare"].mean())


#Embarked

train["Embarked"].fillna("S", inplace = True)

### 2.2 Bining Categorical variables


* Before we fit the data into a machine learning algorithm, there is a step very crucial is that we make sure to encode categorical variables

  correctly We will change Sex to binary, as either 1 for female and 0 for male. We do the same for Embarked. We do this same process on  

  both the training and testing set to prepare our data for Machine Learning.

* **Preprocessing Sex**

In [ ]:
train.loc[train["Sex"] == "male" , "Sex"] = 0
train.loc[train["Sex"] == "female","Sex"] = 1


test.loc[test["Sex"] == "male", "Sex"] = 0
test.loc[test["Sex"] == "female", "Sex"] = 1

* **Processing Embarked**

In [ ]:
train.loc[train["Embarked"] == "S", "Embarked"] = 0
train.loc[train["Embarked"] == "C", "Embarked"] = 1
train.loc[train["Embarked"] == "Q", "Embarked"] = 2


test.loc[test["Embarked"]  == "S", "Embarked"] = 0
test.loc[test["Embarked"]  == "C", "Embarked"] = 1
test.loc[test["Embarked"]  == "Q", "Embarked"] = 2

### 2.3 Creating New Features

- Then, introducing new features as Family size (to join these Parch and SibSp)

* **Family size**

In [ ]:
train["FamSize"] = train["SibSp"] + train["Parch"] + 1
test["FamSize"]  =  test["SibSp"] + test["Parch"]  + 1

- The next option is to cerate IsAlone feature to check wheter a person traveling alolne is more likely to survived or died

* **IsAlone**

In [ ]:
train["IsAlone"] = train.FamSize.apply(lambda x: 1 if x == 1 else 0)
test["IsAlone"]  = test.FamSize.apply( lambda x: 1 if x == 1 else 0)

* **Extraction the passengers titles**

* If we have a quick look in the names of the passengers we will notice that each name has a title in it, so it can be a useful information 

  for our analyze. Therefore we can extract this title from the name of each passenger and then encode it like we did for Sex and Embarked.

In [ ]:
for name in train["Name"]:
    train["Title"] = train["Name"].str.extract("([A-Za-z]+)\.",expand=True)
    
for name in test["Name"]:
    test["Title"] = test["Name"].str.extract("([A-Za-z]+)\.",expand=True)
    
title_replacements = {"Mlle": "Other", "Major": "Other", "Col": "Other", "Sir": "Other", "Don": "Other", "Mme": "Other",
          "Jonkheer": "Other", "Lady": "Other", "Capt": "Other", "Countess": "Other", "Ms": "Other", "Dona": "Other", "Rev": "Other", "Dr": "Other"}

train.replace({"Title": title_replacements}, inplace=True)
test.replace({"Title": title_replacements}, inplace=True)

train.loc[train["Title"] == "Miss", "Title"] = 0
train.loc[train["Title"] == "Mr", "Title"] = 1
train.loc[train["Title"] == "Mrs", "Title"] = 2
train.loc[train["Title"] == "Master", "Title"] = 3
train.loc[train["Title"] == "Other", "Title"] = 4

test.loc[test["Title"] == "Miss", "Title"] = 0
test.loc[test["Title"] == "Mr", "Title"] = 1
test.loc[test["Title"] == "Mrs", "Title"] = 2
test.loc[test["Title"] == "Master", "Title"] = 3
test.loc[test["Title"] == "Other", "Title"] = 4

In [ ]:
print(set(train["Title"]))

### 2.4 Removing irrelevant variables

* The next step is dropping the less relevant features because, The problem with less important features is that they create more noise
 
  and actually take over the importance of real features like Sex and Pclass.

In [ ]:
features_drop = ['Ticket', 'SibSp', 'Parch', "Name", "Cabin", "Fare", "PassengerId"]

train = train.drop(features_drop, axis=1)

test = test.drop(features_drop, axis=1)

### 2.5 Creating dummy variables

In [ ]:
train = pd.get_dummies(train, columns=['Pclass','Sex','Embarked','Title'], 
                       drop_first=False)

test = pd.get_dummies(test, columns=['Pclass','Sex','Embarked','Title'],
                      drop_first=False)

## 3 Pre-Modeling Tasks

### 3.1 Separating the independant and the dependant variable

In [ ]:
X = train.drop('Survived', axis=1)

y = train['Survived']

X.shape,y.shape

### 3.2 Splitting the training data 


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2, random_state= 1)

## 4- Modeling 

* **Training the Random Forest model** 

In [ ]:
RF = RandomForestClassifier(criterion='gini',
                                           n_estimators=1750,
                                           max_depth=7,
                                           min_samples_split=6,
                                           min_samples_leaf=6,
                                           max_features='auto',
                                           oob_score=True,
                                           random_state=42,
                                           n_jobs=-1,
                                           verbose=1) 
RF.fit(X_train, y_train)

* **Predict our model**

In [ ]:
y_pred_train = RF.predict(X_train)

y_pred_test = RF.predict(X_test)

* **Accuracy of the model**

In [ ]:
accu = RF.score(X_train, y_train)
print( "Model Prediction Score", (accu * 100).round(2))

* **Training Accuracy\ Testing Accuracy**

In [ ]:
print("Training accuracy: ", accuracy_score(y_train, y_pred_train))
print("Testing accuracy: ", accuracy_score(y_test, y_pred_test))

### 5- Evaluating the performance of the model

* **Confusion Matrix**

![](https://i1.wp.com/interviewbubble.com/wp-content/uploads/2019/03/1pOtBHai4jFd-ujaNXPilRg.png?resize=936%2C340&ssl=1)

In [ ]:
cm = np.array(confusion_matrix(y_test, y_pred_test, labels=[1,0]))

confusion_mat= pd.DataFrame(cm, index = ["Not-Survived", "Survived"],
                           columns =["Predicted Not Survived", "Predicted Survived"])

confusion_mat

In [ ]:
sns.heatmap(cm,annot=True,fmt='g',cmap='Set3')

* **Accuracy_score**

In [ ]:
accuracy_score(y_test, y_pred_test)

* **Precision_score**

In [ ]:
precision_score(y_test, y_pred_test)

* **classification_report**

In [ ]:
print(classification_report(y_test, y_pred_test))

*** ROC Curve**

![](https://glassboxmedicine.files.wordpress.com/2019/02/roc-curve-v2.png?w=576)

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_pred_test)
roc_auc = auc(fpr, tpr)
print("\nROC AUC on evaluation set",roc_auc )

In [ ]:
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc,color=custom_colors[0])
plt.plot([0, 1], [0, 1], 'k--',color=custom_colors[1])
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

### Submission

In [ ]:
passenger_IDs = pd.read_csv("/kaggle/input/titanic/test.csv")[["PassengerId"]].values
preds = RF.predict(test.values)
preds

In [ ]:
df = {'PassengerId': passenger_IDs.ravel(), 'Survived': preds}
df_predictions = pd.DataFrame(df).set_index(['PassengerId'])
df_predictions.head(10)

In [ ]:
df_predictions.to_csv('/kaggle/working/Predictions.csv')

# Useful resources

- [How to Build a Machine Learning Model](https://towardsdatascience.com/how-to-build-a-machine-learning-model-439ab8fb3fb1)

- [How to find optimal parameters using GridSearchCV?](https://www.dezyre.com/recipes/find-optimal-parameters-using-gridsearchcv)